### https://scikit-learn.org/stable/auto_examples/cross_decomposition/plot_pcr_vs_pls.html

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
tables = './tables'

In [3]:
new_df = pd.read_csv(f'{tables}/active_regr.csv')
new_df

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Ki
0,10.337455,10.337455,0.057778,-1.272753,0.255933,23.375000,505.329,484.161,504.075680,168,...,0,0,0,0,0,0,0,0,0,39.7
1,10.335047,10.335047,0.059736,-1.271941,0.255933,23.375000,505.329,484.161,504.075680,168,...,0,0,0,0,0,0,0,0,0,502.0
2,10.333078,10.333078,0.061184,-1.271281,0.255933,23.375000,505.329,484.161,504.075680,168,...,0,0,0,0,0,0,0,0,0,150.0
3,6.090409,6.090409,0.226017,0.226017,0.547921,12.285714,362.614,353.542,360.961752,104,...,0,0,0,0,0,0,0,0,0,1.0
4,10.328997,10.328997,0.042932,-1.278885,0.279243,23.375000,460.878,439.710,460.126195,168,...,0,0,0,0,0,0,0,0,0,390.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2771,10.207113,10.207113,0.176268,-1.612007,0.403557,26.640000,363.450,342.282,363.136281,134,...,0,0,0,0,0,0,0,0,0,630.0
2772,11.212685,11.212685,0.209838,-0.978555,0.322488,15.540541,521.607,494.391,521.195757,192,...,0,0,0,0,0,0,0,0,0,2.2
2773,11.747357,11.747357,0.187560,-3.317645,0.597420,11.880000,355.379,342.275,355.073910,126,...,0,1,0,0,0,0,0,0,0,110.0
2774,11.566178,11.566178,0.123217,-3.222610,0.585306,11.884615,369.406,354.286,369.089560,132,...,0,1,0,0,0,0,0,0,0,340.0


In [4]:
X = new_df.iloc[:,:-1]
y = new_df['Ki']

In [6]:
def filter_data(X):

    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
    X.drop(to_drop, axis=1, inplace=True)
    
    threshold = 0.9
    sel = VarianceThreshold(threshold=(threshold * (1 - threshold)))
    X = sel.fit_transform(X)
    return X

X = filter_data(X)

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RepeatedKFold, KFold
from sklearn.metrics import mean_squared_error, r2_score